In [1]:
import feedWebGL2.feedback as fd
import numpy as np
fd.widen_notebook()
np.set_printoptions(precision=4)

In [2]:
n = 10
initial_array = np.arange(n) * 0.1
initial_array

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [3]:
shader_GLSL_code = """#version 300 es

        // reverse the input array (in tex1) and add 1.0

        // per vertex input (not used, but at least one is required?)
        in float dummy_input;

        // the sampler -- only the red component contains the float data
        uniform sampler2D tex1;

        out float reversed_value;  // feedback output

        void main() {
            // foil the optimizer -- use the dummy input so it isn't removed (that might cause errors downstream on Firefox)
            gl_Position = vec4(dummy_input,dummy_input,dummy_input,dummy_input);
            // get the sampler size
            ivec2 tsize = textureSize(tex1, 0);
            int reversed_index = tsize[0] - gl_VertexID - 1;
            ivec2 reversed_position = ivec2(reversed_index, 0);
            // get the indexed color
            vec4 redcolor = texelFetch(tex1, reversed_position, 0);
            reversed_value = redcolor.r + 1.0;
        }
"""

feedback_program = fd.FeedbackProgram(
    program = fd.Program(
        vertex_shader = shader_GLSL_code,
        feedbacks = fd.Feedbacks(
            reversed_value = fd.Feedback(num_components=1),
        ),
    ),
    runner = fd.Runner(
        vertices_per_instance = n,
        inputs = fd.Inputs(
            dummy_input = fd.Input(
                num_components = 1,
                from_buffer = fd.BufferLocation(
                    name = "output_buffer",
                ),
            ),
        ),
        samplers = fd.Samplers(
            tex1 = fd.Sampler(
                dim= "2D",
                from_texture= "texture1",
            ),
        ),
    ),
    context = fd.Context(
        buffers = fd.Buffers(
            output_buffer = fd.Buffer(
                array = list(initial_array)
            ),
        ),
        textures= fd.Textures(
            texture1= fd.Texture(
                #type= "FLOAT",
                #format= "RED",  # put the matrix entry in the red component
                #internal_format= "R32F",
                height= 1,
                width=  n,
                array= list(initial_array),
            ),
        ),
    ),
)

# display the widget and debugging information
feedback_program.debugging_display()

In [4]:
#feedback_program.run()

feedback_program.js_init("""

    element.feedback_loop = function () {
        // The main feedback loop operation is written in Javascript to reduce communications overhead.
        element.run_feedback_program();
        // afterward the run link the feedback output on the GPU to the input texture
        var runner = element.feedback_runner;
        var context = element.feedback_context;
        var texture = context.textures.texture1;
        runner.copy_feedback_to_buffer("reversed_value", "output_buffer");
        var from_buffer = context.buffers.output_buffer;
        texture.reload_from_buffer(from_buffer)
    };
""")

In [5]:
feedback_program.element.feedback_loop()
initial_array

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [7]:
r = feedback_program.get_feedback("reversed_value")
r

[[1.899999976158142],
 [1.7999999523162842],
 [1.7000000476837158],
 [1.600000023841858],
 [1.5],
 [1.399999976158142],
 [1.2999999523162842],
 [1.2000000476837158],
 [1.100000023841858],
 [1]]

In [8]:
np.allclose(np.array(list(reversed(r))).ravel() - 1.0, initial_array)

True

In [9]:
feedback_program.element.feedback_loop()
feedback_program.get_feedback("reversed_value")

[[2],
 [2.0999999046325684],
 [2.200000047683716],
 [2.299999952316284],
 [2.4000000953674316],
 [2.5],
 [2.5999999046325684],
 [2.700000047683716],
 [2.799999952316284],
 [2.9000000953674316]]